# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 13** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez </br>
**Team**: Par de Dos </br>
**Members**: Diego Orozco and Aarón Ortega


### FindSpark

In [ ]:
import findspark
findspark.init()

### Connection with SparkCluster

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab13_ParDeDos") \
    .master("spark://368ad5a83fd7:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 05:34:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Schema and DataFrame Creation

In [3]:
from team_ParDeDos.spark_utils import SparkUtils as SpU

schema = SpU.generate_schema([
  ("Alcohol", "double"),
  ("Malic_Acid", "double"),
  ("Ash", "double"),
  ("Ash_Alcanity", "float"),
  ("Magnesium", "integer"),
  ("Total_Phenols", "double"),
  ("Flavanoids", "double"),
  ("Nonflavanoid_Phenols", "double"),
  ("Proanthocyanins", "double"),
  ("Color_Intensity", "double"),
  ("Hue", "double"),
  ("0D280", "double"),
  ("Proline", "integer")
])

wine_df = spark \
          .read \
          .schema(schema) \
          .option("header", "true") \
          .option("mode", "dropMalformed") \
          .csv("/home/jovyan/notebooks/data/wine-clustering.csv")

### Assemble the features into a single vector column

In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["Alcohol", "Malic_Acid", "Ash", "Ash_Alcanity", "Magnesium", "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols", "Proanthocyanins", "Color_Intensity", "Hue", "0D280", "Proline"], outputCol="features")

assembled_df = assembler.transform(wine_df)

### KMeans Function

In [8]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans

def train_and_evaluate_kmeans(df, k, seed=19):
  kmeans = KMeans().setK(k).setSeed(seed)
  model = kmeans.fit(df)
  predictions = model.transform(df)
  evaluator = ClusteringEvaluator()
  silhouette = evaluator.evaluate(predictions)

  print(f"\nK = {k}")
  print(f"Silhouette Score: {silhouette}")
  print("Cluster Centers:")
  for center in model.clusterCenters():
    print(center)
    
  return silhouette


### K = 2

In [9]:
silhouette_k2 = train_and_evaluate_kmeans(assembled_df, k=2)


K = 2
Silhouette Score: 0.8213603513334489
Cluster Centers:
[1.27028455e+01 2.54455285e+00 2.33910569e+00 2.04081301e+01
 9.68130081e+01 2.06211382e+00 1.64146341e+00 3.92682927e-01
 1.45406504e+00 4.85138211e+00 9.08617886e-01 2.40821138e+00
 5.65869919e+02]
[1.36665455e+01 1.87072727e+00 2.42781818e+00 1.74527272e+01
 1.06290909e+02 2.81618182e+00 2.89654545e+00 2.92909091e-01
 1.89690909e+00 5.52036364e+00 1.06665455e+00 3.06672727e+00
 1.15172727e+03]


### K = 10

In [10]:
silhouette_k10 = train_and_evaluate_kmeans(assembled_df, k=10)


K = 10
Silhouette Score: 0.609991127409139
Cluster Centers:
[1.39289474e+01 1.78157895e+00 2.48684211e+00 1.70789474e+01
 1.05315789e+02 2.90578947e+00 3.09947368e+00 2.84736842e-01
 1.93789474e+00 6.37105263e+00 1.11000000e+00 3.00894737e+00
 1.36747368e+03]
[1.26437931e+01 2.80896552e+00 2.26724138e+00 2.07241379e+01
 9.27586207e+01 1.80034483e+00 1.35379310e+00 4.06206897e-01
 1.34310345e+00 4.53482759e+00 8.73793103e-01 2.26344828e+00
 5.29827586e+02]
[1.32000000e+01 2.87375000e+00 2.46000000e+00 1.94875001e+01
 1.13375000e+02 2.14125000e+00 1.78875000e+00 3.67500000e-01
 1.60375000e+00 6.25500000e+00 8.25000000e-01 2.57750000e+00
 8.34375000e+02]
[ 12.33823529   2.39117647   2.22352941  20.52941176  90.17647059
   2.33823529   2.25823529   0.35411765   1.59470588   2.79470588
   0.96941176   2.95411765 347.70588235]
[1.27796667e+01 2.36866667e+00 2.42766667e+00 2.05100000e+01
 9.66000000e+01 2.03866667e+00 1.36800000e+00 4.14000000e-01
 1.34733333e+00 5.82066667e+00 8.52000000e-0

### K = 15

In [11]:
silhouette_k15 = train_and_evaluate_kmeans(assembled_df, k=15)


K = 15
Silhouette Score: 0.666330934764843
Cluster Centers:
[1.38278571e+01 1.74214286e+00 2.53428571e+00 1.76000001e+01
 1.06214286e+02 2.75928571e+00 2.90500000e+00 3.05714286e-01
 1.77642857e+00 5.93214286e+00 1.11857143e+00 3.00000000e+00
 1.28821429e+03]
[1.25341667e+01 2.25666667e+00 2.41250000e+00 2.25249999e+01
 9.22500000e+01 1.90083333e+00 1.86833333e+00 4.38333333e-01
 1.50416667e+00 4.84749992e+00 9.07500000e-01 2.42666667e+00
 4.79083333e+02]
[1.419e+01 1.590e+00 2.480e+00 1.650e+01 1.080e+02 3.300e+00 3.930e+00
 3.200e-01 1.860e+00 8.700e+00 1.230e+00 2.820e+00 1.680e+03]
[1.313500e+01 2.833750e+00 2.338125e+00 1.880625e+01 1.090000e+02
 2.163750e+00 1.720625e+00 3.643750e-01 1.635625e+00 5.208125e+00
 9.325000e-01 2.550000e+00 7.466875e+02]
[1.26640000e+01 3.24000000e+00 2.39533333e+00 2.15333333e+01
 9.66000000e+01 1.85600000e+00 1.22600000e+00 4.24666667e-01
 1.27466667e+00 4.67533333e+00 8.64666667e-01 2.19533333e+00
 5.73133333e+02]
[1.30300000e+01 2.43785714e+00 2.

### K = 20

In [12]:
silhouette_k20 = train_and_evaluate_kmeans(assembled_df, k=20)


K = 20
Silhouette Score: 0.6341822740887433
Cluster Centers:
[1.38278571e+01 1.74214286e+00 2.53428571e+00 1.76000001e+01
 1.06214286e+02 2.75928571e+00 2.90500000e+00 3.05714286e-01
 1.77642857e+00 5.93214286e+00 1.11857143e+00 3.00000000e+00
 1.28821429e+03]
[1.26077778e+01 2.49222222e+00 2.24500000e+00 2.02500000e+01
 9.15000000e+01 1.76000000e+00 1.40000000e+00 4.02222222e-01
 1.29333333e+00 4.42777778e+00 8.91111111e-01 2.27555556e+00
 5.09166667e+02]
[1.40200000e+01 1.89666667e+00 2.34666667e+00 1.63666668e+01
 1.04333333e+02 3.03666667e+00 3.38333333e+00 2.43333333e-01
 2.43000000e+00 6.71666667e+00 1.18000000e+00 3.01666667e+00
 1.52400000e+03]
[1.32153846e+01 3.05692308e+00 2.40000000e+00 1.91076923e+01
 1.11076923e+02 2.08000000e+00 1.50307692e+00 3.82307692e-01
 1.58923077e+00 5.38461538e+00 8.87692308e-01 2.40923077e+00
 7.48307692e+02]
[1.27000e+01 2.18625e+00 2.37375e+00 2.01875e+01 1.04750e+02 1.93750e+00
 1.21250e+00 3.55000e-01 1.25875e+00 5.45250e+00 7.86250e-01 2.00

### All results

In [13]:
results = {
    2: silhouette_k2,
    10: silhouette_k10,
    15: silhouette_k15,
    20: silhouette_k20
}

for k, score in results.items():
    print(f"K = {k} -> Silhouette Score = {score}")


K = 2 -> Silhouette Score = 0.8213603513334489
K = 10 -> Silhouette Score = 0.609991127409139
K = 15 -> Silhouette Score = 0.666330934764843
K = 20 -> Silhouette Score = 0.6341822740887433


In [14]:
sc.stop()